In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, learning_curve
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#-------------------Escoger modelo con ajuste de hyperparámetros con GridSearchCV y RandomizedSearchCV

pipeline_ridge = Pipeline([
    ('scaler', StandardScaler()), # Primer paso: escalar los datos
    ('ridge', Ridge())           # Segundo paso: aplicar el modelo Ridge
])

# Definir el rango de hiperparámetros para GridSearchCV
# Para Ridge, el hiperparámetro principal es 'alpha'
# El nombre del parámetro en el pipeline es <nombre_del_paso>__<nombre_del_parametro>
param_grid_ridge = {
    'ridge__alpha': [0.001, 0.01, 0.1, 1, 10, 100] # Rango de valores de alpha a probar
}

# Definir el rango de hiperparámetros para RandomizedSearchCV
# Puedes usar una distribución si el espacio de búsqueda es muy grande
param_dist_ridge = {
    'ridge__alpha': np.random.uniform(0.001, 100, 20) # 20 valores aleatorios entre 0.001 y 100
}


print("Aplicando GridSearchCV para Ridge...")
# Inicializar GridSearchCV
# cv=5 significa validación cruzada de 5 pliegues
# scoring='neg_mean_squared_error' se usa porque GridSearchCV por defecto maximiza el score,
# y queremos minimizar el MSE, por lo tanto usamos el negativo.
# n_jobs=-1 usa todos los núcleos disponibles
grid_search_ridge = GridSearchCV(estimator=pipeline_ridge,
                                 param_grid=param_grid_ridge,
                                 cv=5,
                                 scoring='neg_mean_squared_error',
                                 n_jobs=-1)

# Entrenar GridSearchCV en los datos de entrenamiento
# Esto realizará la validación cruzada para cada combinación de hiperparámetros
grid_search_ridge.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params_grid_ridge = grid_search_ridge.best_params_
print(f"Mejores hiperparámetros encontrados con GridSearchCV para Ridge: {best_params_grid_ridge}")

# Obtener el mejor modelo encontrado por GridSearchCV
best_ridge_model_grid = grid_search_ridge.best_estimator_

# Evaluar el mejor modelo en el conjunto de prueba
y_pred_ridge_grid = best_ridge_model_grid.predict(X_test)
rmse_ridge_grid = np.sqrt(mean_squared_error(y_test, y_pred_ridge_grid))
r2_ridge_grid = r2_score(y_test, y_pred_ridge_grid)

print(f"Resultados del mejor modelo Ridge (GridSearchCV) en el conjunto de prueba:")
print(f"  RMSE: {rmse_ridge_grid:.4f}")
print(f"  R²: {r2_ridge_grid:.4f}")

# Graficar la comparación para el mejor modelo Ridge (GridSearchCV)
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred_ridge_grid, alpha=0.5, label='Observaciones (Valor Real vs Predicho)')
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2, label='Línea Ideal (Real = Predicho)')
plt.title(f'Valor real vs Valor predicho - Ridge (GridSearchCV)')
plt.xlabel('Valor real')
plt.ylabel('Valor predicho')
plt.xlim(y.min(), y.max())
plt.ylim(y.min(), y.max())
plt.grid()
plt.legend()
plt.show()


print("\nAplicando RandomizedSearchCV para Ridge...")
# Inicializar RandomizedSearchCV
# n_iter=20 significa que probará 20 combinaciones aleatorias
# cv=5 significa validación cruzada de 5 pliegues
# scoring='neg_mean_squared_error'
# n_jobs=-1 usa todos los núcleos disponibles
# random_state para reproducibilidad
random_search_ridge = RandomizedSearchCV(estimator=pipeline_ridge,
                                         param_distributions=param_dist_ridge,
                                         n_iter=20, # Número de combinaciones a probar
                                         cv=5,
                                         scoring='neg_mean_squared_error',
                                         n_jobs=-1,
                                         random_state=42)

# Entrenar RandomizedSearchCV en los datos de entrenamiento
random_search_ridge.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params_random_ridge = random_search_ridge.best_params_
print(f"Mejores hiperparámetros encontrados con RandomizedSearchCV para Ridge: {best_params_random_ridge}")

# Obtener el mejor modelo encontrado por RandomizedSearchCV
best_ridge_model_random = random_search_ridge.best_estimator_

# Evaluar el mejor modelo en el conjunto de prueba
y_pred_ridge_random = best_ridge_model_random.predict(X_test)
rmse_ridge_random = np.sqrt(mean_squared_error(y_test, y_pred_ridge_random))
r2_ridge_random = r2_score(y_test, y_pred_ridge_random)

print(f"Resultados del mejor modelo Ridge (RandomizedSearchCV) en el conjunto de prueba:")
print(f"  RMSE: {rmse_ridge_random:.4f}")
print(f"  R²: {r2_ridge_random:.4f}")

# Graficar la comparación para el mejor modelo Ridge (RandomizedSearchCV)
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred_ridge_random, alpha=0.5, label='Observaciones (Valor Real vs Predicho)')
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2, label='Línea Ideal (Real = Predicho)')
plt.title(f'Valor real vs Valor predicho - Ridge (RandomizedSearchCV)')
plt.xlabel('Valor real')
plt.ylabel('Valor predicho')
plt.xlim(y.min(), y.max())
plt.ylim(y.min(), y.max())
plt.grid()
plt.legend()
plt.show()

results['Ridge_GridSearchCV'] = {'RMSE': rmse_ridge_grid, 'R²': r2_ridge_grid}
results['Ridge_RandomizedSearchCV'] = {'RMSE': rmse_ridge_random, 'R²': r2_ridge_random}
print("\nResumen de resultados:")
print(results)



#-----------------------------Curvas de Aprendizaje Ridge-----------------------------
X = data_social.drop('actual_productivity_score', axis=1).select_dtypes(include=np.number)
y = data_social['actual_productivity_score']

best_ridge_model = grid_search_ridge.best_estimator_
linear_model = LinearRegression()

models_for_learning_curve = {
    'Ridge (Optimizado)': best_ridge_model
}

for name, estimator in models_for_learning_curve.items():
    print(f"Generando curva de aprendizaje para {name}...")

    # Calcula los puntos para la curva de aprendizaje
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=5, scoring='neg_mean_squared_error', n_jobs=-1,
        train_sizes=np.linspace(0.1, 1.0, 10) # Genera 10 puntos entre 10% y 100% del tamaño del entrenamiento
    )

    # Convertir los scores de MSE negativo a RMSE (Raíz del Error Cuadrático Medio)
    # RMSE = sqrt(-MSE)
    train_scores_rmse = np.sqrt(-train_scores)
    test_scores_rmse = np.sqrt(-test_scores)
    # Calcular la media y desviación estándar de los scores para cada tamaño de entrenamiento
    train_scores_mean = np.mean(train_scores_rmse, axis=1)
    train_scores_std = np.std(train_scores_rmse, axis=1)
    test_scores_mean = np.mean(test_scores_rmse, axis=1)
    test_scores_std = np.std(test_scores_rmse, axis=1)

    # Graficar la curva de aprendizaje
    plt.figure(figsize=(5, 3))
    plt.title(f"Curva de Aprendizaje - {name}")
    plt.xlabel("Tamaño del Conjunto de Entrenamiento")
    plt.ylabel("RMSE") # Usamos RMSE en el eje Y ya que es más interpretable

    # Graficar la curva de entrenamiento con banda de error
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Puntuación Entrenamiento")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Puntuación Validación Cruzada")

    plt.legend(loc="best")
    plt.ylim(bottom=0) # Asegura que el eje Y comience en 0 o más

    plt.show()



#Curva de aprendizaje implícita
#El flujo sugiere una curva estable para Ridge:

#Buen rendimiento en entrenamiento y prueba

#Baja varianza entre distintos modelos
#Conclusión: El modelo no está ni subajustado ni sobreajustado (overfitting ni underfitting)
#y se encuentra en una zona óptima de la curva de aprendizaje. Esto indica un aprendizaje efectivo,
# donde agregar más datos no necesariamente mejorará mucho el modelo sin cambiar su estructura.


Aplicando GridSearchCV para Ridge...


NameError: name 'X_train' is not defined